In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

# Autenticación e inicialización
ee.Authenticate()
ee.Initialize(project='my-project-unimagdalena')

# Área de interés (Cereté, Córdoba)
aoi = ee.Geometry.BBox(-75.89, 8.88, -75.86, 8.90)

# Año de análisis
year = 2023
months = list(range(1, 13))

# Función para calcular NDVI mensual
def get_monthly_ndvi(month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')

    # MODIS
    modis = (
        ee.ImageCollection('MODIS/006/MOD13Q1')
        .filterDate(start, end)
        .filterBounds(aoi)
        .select('NDVI')
    )

    # Sentinel-2
    s2 = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(start, end)
        .filterBounds(aoi)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .select(['B4', 'B8'])
    )

    # Get counts of images in collections
    modis_count = modis.size()
    s2_count = s2.size()

    # Initialize values as None
    modis_value = None
    s2_value = None

    # Calculate MODIS mean and reduce only if the collection is not empty
    if modis_count.getInfo() > 0: # Use getInfo() to resolve the size server-side
        modis_mean = modis.mean().multiply(0.0001).rename('NDVI_MODIS')
        modis_value = modis_mean.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=250,
            maxPixels=1e9
        ).get('NDVI_MODIS')

    # Calculate Sentinel-2 NDVI and reduce only if the collection is not empty
    if s2_count.getInfo() > 0: # Use getInfo() to resolve the size server-side
        ndvi_s2 = s2.median().normalizedDifference(['B8', 'B4']).rename('NDVI_S2')
        s2_value = ndvi_s2.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=10,
            maxPixels=1e9
        ).get('NDVI_S2')


    return ee.Feature(None, {
        'month': month,
        'NDVI_MODIS': modis_value,
        'NDVI_S2': s2_value
    })

# Crear FeatureCollection mensual
features = ee.FeatureCollection([get_monthly_ndvi(m) for m in months])

# 🔁 Alternativa manual: convertir a lista de diccionarios
fc_info = features.getInfo()
features_list = [f['properties'] for f in fc_info['features']]

# Convertir a DataFrame
data = pd.DataFrame(features_list)
data['month'] = data['month'].astype(int)

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(data['month'], data['NDVI_MODIS'], label='MODIS (250 m)', marker='o', color='green')
plt.plot(data['month'], data['NDVI_S2'], label='Sentinel-2 (10 m)', marker='o', color='darkgreen')

plt.title(f'Comparación mensual del NDVI (MODIS vs Sentinel-2) - {year}')
plt.xlabel('Mes')
plt.ylabel('NDVI')
plt.xticks(months)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

data['diferencia'] = data['NDVI_S2'] - data['NDVI_MODIS']
print("Diferencia promedio S2 - MODIS:", data['diferencia'].mean())